## 📓 03_prompt_engineering.ipynb

# 🎯 Prompt Engineering for Helsefaglige Oppgaver

## Læringsmål
- Mestre grunnleggende prompt-teknikker
- Lære few-shot learning for medisinske caser
- Implementere chain-of-thought for komplekse vurderinger

In [73]:
import os
from openai import OpenAI
from typing import List, Dict
import json
from dotenv import load_dotenv

# Last miljøvariabler fra .env fil
load_dotenv()

# Sjekk at API-nøkkel er lastet
if os.getenv("OPENAI_API_KEY"):
    print("✅ OPENAI_API_KEY funnet i miljøvariabler")
    # Vis bare de første og siste tegnene for sikkerhet
    key = os.getenv("OPENAI_API_KEY")
    print(f"   Nøkkel: {key[:7]}...{key[-4:]}")
else:
    print("❌ OPENAI_API_KEY ikke funnet")

# Initialiser klient (bruker miljøvariabel OPENAI_API_KEY)
client = OpenAI()

print("🎯 Prompt Engineering - Kommuniser effektivt med AI")
print("✅ OpenAI klient initialisert!")

# Dersom du ikke har python-dotenv installert, installér den først:
# !pip install python-dotenv

✅ OPENAI_API_KEY funnet i miljøvariabler
   Nøkkel: sk-proj...ypoA
🎯 Prompt Engineering - Kommuniser effektivt med AI
✅ OpenAI klient initialisert!


## Grunnleggende prompt-prinsipper

1. **Vær spesifikk**: Klar kontekst og instruksjoner
2. **Gi eksempler**: Few-shot learning
3. **Strukturer output**: Be om spesifikt format
4. **Tenk stegvis**: Chain-of-thought

In [74]:
def send_prompt(prompt: str, temperature: float = 0.7) -> str:
    """Hjelpefunksjon for å sende prompts til GPT"""
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Simulert respons (API ikke tilgjengelig): {e}"

## Teknikk 1: Zero-shot vs Few-shot

In [75]:
print("🎯 FORSKJELLER: Når few-shot faktisk hjelper")
print("=" * 70)

# Her er cases hvor few-shot VIRKELIG gjør en forskjell:
# 1. Terminologi/sjargong
# 2. Kontekstuelle nyanser  
# 3. Edge cases som krever spesifikk kunnskap

# EKSEMPEL 1: Medisinsk dokumentasjon vs. pasientspråk
print("\n📝 EKSEMPEL 1: Tolkning av pasientspråk")
print("=" * 50)

patient_language_cases = [
    "Jeg har vondt i brystet når jeg går i trapper, men det går over når jeg hviler",
    "Det banker og dunker i brystet mitt, spesielt når jeg blir stresset",
    "Jeg får ikke pust når jeg ligger ned, må sove med mange puter",
    "Beina mine hovner opp utover dagen, særlig når det er varmt"
]

# Zero-shot: Generell klassifisering
zero_shot_patient = """
Klassifiser følgende pasientbeskrivelse som 'Akutt' eller 'Ikke-akutt':
{beskrivelse}
"""

# Few-shot: Med pasientspråk-eksempler
few_shot_patient = """
Klassifiser pasientbeskrivelser som 'Akutt' eller 'Ikke-akutt'. 
Pasienter bruker ofte hverdagsspråk istedenfor medisinsk terminologi:

Eksempel 1: "Det banker i brystet" = palpitasjoner → Ikke-akutt (hvis ikke andre symptomer)
Eksempel 2: "Får ikke pust når jeg ligger" = ortopné → Akutt (hjertesykdom-mistanke)
Eksempel 3: "Vondt i brystet ved trapper" = anstrengelsessmerte → Akutt (angina-mistanke)
Eksempel 4: "Beina hovner" = ødemer → Ikke-akutt (hvis kronisk)

Klassifiser: {beskrivelse}
Svar: Akutt eller Ikke-akutt
"""

for i, case in enumerate(patient_language_cases, 1):
    print(f"\nCase {i}: {case}")
    
    zero_result = send_prompt(zero_shot_patient.format(beskrivelse=case), temperature=0)
    few_result = send_prompt(few_shot_patient.format(beskrivelse=case), temperature=0)
    
    print(f"Zero-shot: {zero_result}")
    print(f"Few-shot:  {few_result}")

# EKSEMPEL 2: Kulturelle og sosiale faktorer
print("\n\n🌍 EKSEMPEL 2: Kulturelle nyanser og sosial kontekst")
print("=" * 50)

cultural_cases = [
    "Eldre somalisk kvinne som ikke vil forklare underlivssmerter til mannlig lege",
    "Samisk pasient som bruker tradisjonell medisin sammen med moderne behandling",
    "Tolk-avhengig pasient som virker redd og unnvikende under konsultasjon",
    "Pasient fra kollektivistisk kultur som insisterer på familiemedlemmer tilstede"
]

zero_shot_cultural = """
Vurder om denne situasjonen krever 'Øyeblikkelig oppmerksomhet' eller 'Kan vente':
{situasjon}
"""

few_shot_cultural = """
Vurder medisinske situasjoner med kulturell sensitivitet. Kulturelle faktorer påvirker pasientopplevelse:

Eksempel 1: "Kvinne vil ikke snakke med mannlig lege om underlivs-problemer" 
→ Kan vente (tilby kvinnelig lege, respekter kulturelle behov)

Eksempel 2: "Tolk-avhengig pasient virker redd og unnvikende"
→ Øyeblikkelig oppmerksomhet (kan skjule alvorlige symptomer pga språkbarriere)

Eksempel 3: "Bruker tradisjonell medisin sammen med moderne"
→ Kan vente (kartlegg interaksjoner, men ikke nødvendigvis akutt)

Vurder: {situasjon}
Svar: Øyeblikkelig oppmerksomhet eller Kan vente
"""

for i, case in enumerate(cultural_cases, 1):
    print(f"\nKulturelt case {i}: {case}")
    
    zero_result = send_prompt(zero_shot_cultural.format(situasjon=case), temperature=0)
    few_result = send_prompt(few_shot_cultural.format(situasjon=case), temperature=0)
    
    print(f"Zero-shot: {zero_result}")
    print(f"Few-shot:  {few_result}")

# EKSEMPEL 3: Spesialisert medisinsk domene
print("\n\n🧠 EKSEMPEL 3: Psykiatriske vurderinger")
print("=" * 50)

psychiatric_cases = [
    "Pasient sier 'jeg hører stemmer som forteller meg å skade andre'",
    "Pasient har ikke sovet på 5 dager, snakker fort og planlegger storslåtte prosjekter",
    "Pasient nekter å spise fordi 'maten er forgiftet av regjeringen'",
    "Pasient rapporterer at 'alle kan lese tankene mine gjennom TV-en'"
]

zero_shot_psych = """
Klassifiser som 'Psykiatrisk akuttsituasjon' eller 'Kan vente på poliklinisk vurdering':
{symptom}
"""

few_shot_psych = """
Psykiatriske akuttsituasjoner krever øyeblikkelig håndtering av sikkerhetshensyn:

AKUTT: Umiddelbar fare for selv/andre
- "Stemmer som befaler skade" → Psykiatrisk akuttsituasjon (kommando-hallusinasjoner)
- "Ikke sovet 5 dager + mani" → Psykiatrisk akuttsituasjon (alvorlig mani)

KAN VENTE: Psykotiske symptomer uten fare
- "Paranoid om mat" → Kan vente (psykose uten aggresjon)
- "Tankeleseideliv via TV" → Kan vente (bizarre vrangforestillinger uten fare)

Klassifiser: {symptom}
Svar: Psykiatrisk akuttsituasjon eller Kan vente på poliklinisk vurdering
"""

for i, case in enumerate(psychiatric_cases, 1):
    print(f"\nPsykiatrisk case {i}: {case}")
    
    zero_result = send_prompt(zero_shot_psych.format(symptom=case), temperature=0)
    few_result = send_prompt(few_shot_psych.format(symptom=case), temperature=0)
    
    print(f"Zero-shot: {zero_result}")
    print(f"Few-shot:  {few_result}")

# KONKLUSJON
print("\n\n📋 KONKLUSJON: Når few-shot virkelig hjelper")
print("=" * 70)
print("✅ GPT-3.5-turbo er allerede svært god på:")
print("   • Grunnleggende medisinske klassifikasjoner")
print("   • Åpenbare red flags")
print("   • Standard akutt/ikke-akutt vurderinger")
print()
print("🎯 Few-shot prompting hjelper mest ved:")
print("   • Tolkning av pasientspråk vs. medisinsk terminologi")
print("   • Kulturelle og sosiale nyanser") 
print("   • Spesialiserte domener (psykiatri, pediatri)")
print("   • Edge cases som krever kontekstspesifikk kunnskap")
print("   • Når du vil ha konsistent resonnering/forklaring")
print()
print("💡 Viktig lærdom:")
print("   Moderne LLM-er er gode på generell medisinsk kunnskap,")
print("   men few-shot hjelper med domene-spesifikke nyanser og")
print("   situasjoner som krever spesifikk klinisk erfaring.")

# Bonus: Vis faktisk nytte av few-shot
print("\n\n🔍 BONUS: Detaljert resonnering")
print("=" * 50)

complex_case = "67 år gammel mann sier 'det føles som en elefant sitter på brystet mitt når jeg går'"

simple_prompt = f"Klassifiser som akutt eller ikke-akutt: {complex_case}"
detailed_prompt = f"""
Du er erfaren kardiolog. Klassifiser og forklar:

Pasientspråk: "elefant på brystet" = klassisk beskrivelse av angina
Utløsende faktor: "når jeg går" = anstrengelsesutløst
Alder: 67 år = høy kardiovaskulær risiko

Case: {complex_case}
Gi klassifisering OG resonnering.
"""

print("Enkel prompt:")
print(send_prompt(simple_prompt, temperature=0))
print("\nDetaljert few-shot prompt:")
print(send_prompt(detailed_prompt, temperature=0))

🎯 FORSKJELLER: Når few-shot faktisk hjelper

📝 EKSEMPEL 1: Tolkning av pasientspråk

Case 1: Jeg har vondt i brystet når jeg går i trapper, men det går over når jeg hviler
Zero-shot: Ikke-akutt
Few-shot:  Akutt

Case 2: Det banker og dunker i brystet mitt, spesielt når jeg blir stresset
Zero-shot: Akutt
Few-shot:  Akutt

Case 3: Jeg får ikke pust når jeg ligger ned, må sove med mange puter
Zero-shot: Akutt
Few-shot:  Akutt

Case 4: Beina mine hovner opp utover dagen, særlig når det er varmt
Zero-shot: Ikke-akutt
Few-shot:  Ikke-akutt


🌍 EKSEMPEL 2: Kulturelle nyanser og sosial kontekst

Kulturelt case 1: Eldre somalisk kvinne som ikke vil forklare underlivssmerter til mannlig lege
Zero-shot: Denne situasjonen krever øyeblikkelig oppmerksomhet. Underlivssmerter kan være et tegn på alvorlig sykdom eller komplikasjoner, og det er viktig å få en grundig undersøkelse av en lege så snart som mulig. Det er også viktig å respektere pasientens ønske om å snakke med en kvinnelig lege, da dette 

In [76]:
print("🎯 STØRRE FORSKJELLER: Zero-shot vs Few-shot")
print("=" * 70)

# EKSEMPEL 1: Pediatriske vitalparametere (krever aldersbasert kunnskap)
print("\n👶 EKSEMPEL 1: Pediatriske vitalparametere")
print("=" * 50)

pediatric_cases = [
    {
        "case": "3 måneder gammel baby: Puls 180, respirasjonsfrekvens 60, temperatur 36.8°C",
        "correct": "Normale verdier for alder",
        "challenge": "Disse verdiene ville vært alarmerende hos voksne"
    },
    {
        "case": "5 år gammel barn: Blodtrykk 85/50, puls 120, våken og aktiv",
        "correct": "Normale verdier for alder", 
        "challenge": "Lavt blodtrykk og høy puls hos voksne = sjokk"
    },
    {
        "case": "12 år gammel: Puls 55, blodtrykk 110/65, trener fotball aktivt",
        "correct": "Normale verdier for aktiv ungdom",
        "challenge": "Bradykardi hos voksne kan være patologisk"
    }
]

🎯 STØRRE FORSKJELLER: Zero-shot vs Few-shot

👶 EKSEMPEL 1: Pediatriske vitalparametere


In [77]:
# Zero-shot uten pediatrisk kunnskap
zero_shot_pediatric = """
Vurder disse vitalparameterne som 'Normale' eller 'Bekymringsfulle':

{case}

Svar kun: Normale eller Bekymringsfulle
"""

In [78]:
# Few-shot med pediatrisk referanseområder
few_shot_pediatric = """
Vurder vitalparametere basert på aldersbaserte normalverdier:

NORMALVERDIER ETTER ALDER:
Spedbarn (0-12 mnd): Puls 100-180, RR 30-60, BT 60-90/30-60
Småbarn (1-5 år): Puls 80-140, RR 20-40, BT 80-110/50-70  
Barn (6-12 år): Puls 60-120, RR 15-25, BT 90-120/60-80
Tenåringer: Puls 50-100, RR 12-20, BT 100-130/60-85

Eksempel 1: "Baby puls 180" → Normale (øvre normalgrense for spedbarn)
Eksempel 2: "5-åring puls 120" → Normale (normalområdet for småbarn)
Eksempel 3: "Tenåring puls 55" → Normale (kan være normalt hos aktive)

Vurder: {case}
Svar: Normale eller Bekymringsfulle
"""

In [79]:
print("PEDIATRISKE VITALPARAMETERE:")
for i, case_data in enumerate(pediatric_cases, 1):
    case = case_data["case"]
    print(f"\nCase {i}: {case}")
    print(f"Utfordring: {case_data['challenge']}")
    
    zero_result = send_prompt(zero_shot_pediatric.format(case=case), temperature=0)
    few_result = send_prompt(few_shot_pediatric.format(case=case), temperature=0)
    
    print(f"Zero-shot: {zero_result}")
    print(f"Few-shot:  {few_result}")
    print(f"Korrekt:   {case_data['correct']}")

PEDIATRISKE VITALPARAMETERE:

Case 1: 3 måneder gammel baby: Puls 180, respirasjonsfrekvens 60, temperatur 36.8°C
Utfordring: Disse verdiene ville vært alarmerende hos voksne
Zero-shot: Bekymringsfulle
Few-shot:  Svaret er bekymringsfullt. Selv om pulsen på 180 er innenfor det normale området for spedbarn, er en respirasjonsfrekvens på 60 høyere enn normalt for spedbarn (30-60). Det kan være lurt å kontakte helsepersonell for videre vurdering. Temperaturen på 36.8°C er innenfor det normale området.
Korrekt:   Normale verdier for alder

Case 2: 5 år gammel barn: Blodtrykk 85/50, puls 120, våken og aktiv
Utfordring: Lavt blodtrykk og høy puls hos voksne = sjokk
Zero-shot: Normale
Few-shot:  Svaret er normale. Barnet har en puls innenfor det normale området for småbarn og et blodtrykk som ligger innenfor normalområdet for denne aldersgruppen. Det faktum at barnet er våken og aktiv er også et godt tegn.
Korrekt:   Normale verdier for alder

Case 3: 12 år gammel: Puls 55, blodtrykk 110/65, 

In [80]:
# EKSEMPEL 2: Medikament-dosering og interaksjoner
print("\n\n💊 EKSEMPEL 2: Medikament-dosering etter nyrefunksjon")
print("=" * 50)

medication_cases = [
    {
        "case": "85 år, kreatinin 180 μmol/L, skal ha Metformin 1000mg x2",
        "risk": "Høy - Metformin kontraindisert ved nedsatt nyrefunksjon",
        "action": "Stopp Metformin"
    },
    {
        "case": "70 år, eGFR 25, får Digoksin 0.25mg daglig i 5 år uten kontroll",
        "risk": "Høy - Digoksin akkumulerer ved dårlig nyrefunksjon", 
        "action": "Doser ned/sjekk digoksin-nivå"
    },
    {
        "case": "45 år, kreatinin 65 μmol/L, skal starte ACE-hemmer",
        "risk": "Lav - Normal nyrefunksjon for alder",
        "action": "OK å starte, monitorér"
    }
]



💊 EKSEMPEL 2: Medikament-dosering etter nyrefunksjon


In [81]:
zero_shot_medication = """
Vurder medikament-sikkerhet som 'Trygt' eller 'Høy risiko':

{case}

Svar kun: Trygt eller Høy risiko
"""

In [82]:
few_shot_medication = """
Vurder medikament-sikkerhet basert på nyrefunksjon og alder:

NYREFUNKSJON & MEDIKAMENTER:
- Kreatinin >130 eller eGFR <30 = alvorlig nedsatt
- Metformin: STOPP ved kreatinin >130 (laktacidose-risiko)
- Digoksin: Akkumulerer ved dårlig nyrefunksjon, halver dose
- ACE-hemmere: OK ved mild nedsettelse, følg med kreatinin

Eksempel 1: "Kreatinin 180 + Metformin" → Høy risiko (kontraindikasjon)
Eksempel 2: "eGFR 25 + Digoksin" → Høy risiko (akkumulering)
Eksempel 3: "Kreatinin 65 + ACE-hemmer" → Trygt (normal funksjon)

Vurder: {case}
Svar: Trygt eller Høy risiko
"""

In [83]:
print("MEDIKAMENT-SIKKERHET:")
for i, case_data in enumerate(medication_cases, 1):
    case = case_data["case"]
    print(f"\nCase {i}: {case}")
    
    zero_result = send_prompt(zero_shot_medication.format(case=case), temperature=0)
    few_result = send_prompt(few_shot_medication.format(case=case), temperature=0)
    
    print(f"Zero-shot: {zero_result}")
    print(f"Few-shot:  {few_result}")
    print(f"Riktig:    {case_data['risk']} - {case_data['action']}")


MEDIKAMENT-SIKKERHET:

Case 1: 85 år, kreatinin 180 μmol/L, skal ha Metformin 1000mg x2
Zero-shot: Høy risiko
Few-shot:  Høy risiko (kontraindikasjon) - Metformin bør stoppes ved kreatinin >130 μmol/L på grunn av risikoen for laktacidose.
Riktig:    Høy - Metformin kontraindisert ved nedsatt nyrefunksjon - Stopp Metformin

Case 2: 70 år, eGFR 25, får Digoksin 0.25mg daglig i 5 år uten kontroll
Zero-shot: Høy risiko
Few-shot:  Høy risiko. Personen har alvorlig nedsatt nyrefunksjon (eGFR 25) og har fått Digoksin i 5 år uten kontroll. Digoksin akkumulerer ved dårlig nyrefunksjon, og det anbefales å halvere dosen i slike tilfeller. Det er viktig å følge opp med jevnlige kontroller for å unngå bivirkninger og komplikasjoner.
Riktig:    Høy - Digoksin akkumulerer ved dårlig nyrefunksjon - Doser ned/sjekk digoksin-nivå

Case 3: 45 år, kreatinin 65 μmol/L, skal starte ACE-hemmer
Zero-shot: Høy risiko
Few-shot:  Svaret er trygt. Med en kreatininverdi på 65 μmol/L og en alder på 45 år, er nyrefu

In [84]:
# EKSEMPEL 3: Komplekse EKG-tolkninger
print("\n\n📈 EKSEMPEL 3: EKG-tolkning og klinisk kontekst")
print("=" * 50)

ecg_cases = [
    {
        "case": "25 år, mannlig idrettsutøver: Bradykardi 45 slag/min, normale intervaller, asymptomatisk",
        "interpretation": "Fysiologisk bradykardi - normalt hos utøvere",
        "action": "Ingen behandling nødvendig"
    },
    {
        "case": "75 år, kvinne med svimmelhet: QRS 140ms, diskordante T-bølger, venstre akse",
        "interpretation": "Venstre grenblokk - kan maskere hjerteinfarkt", 
        "action": "Akutt utredning nødvendig"
    },
    {
        "case": "50 år med brystsmerter: ST-depresjon 1mm i V4-V6 under ergometri-test",
        "interpretation": "Positiv stress-test - iskemi",
        "action": "Videre kardiologisk utredning"
    }
]

zero_shot_ecg = """
Tolkn dette EKG-funnet som 'Normalt/benignt' eller 'Krever oppfølging':

{case}

Svar kun: Normalt/benignt eller Krever oppfølging
"""

few_shot_ecg = """
Tolkn EKG-funn i klinisk kontekst:

EKG-TOLKNING PRINSIPPER:
- Alder og treningsstatus påvirker normalverdier
- Symptomer endrer tolkningen dramatisk  
- Grenblokk kan maskere andre patologiske funn

Eksempel 1: "Ung idrettsutøver + bradykardi" → Normalt/benignt (fysiologisk)
Eksempel 2: "Eldre + grenblokk + symptomer" → Krever oppfølging (kan skjule patologi)
Eksempel 3: "ST-endringer under stress" → Krever oppfølging (iskemi)

Tolkn: {case}
Svar: Normalt/benignt eller Krever oppfølging
"""



📈 EKSEMPEL 3: EKG-tolkning og klinisk kontekst


In [85]:
print("EKG-TOLKNING:")
for i, case_data in enumerate(ecg_cases, 1):
    case = case_data["case"]
    print(f"\nCase {i}: {case}")
    
    zero_result = send_prompt(zero_shot_ecg.format(case=case), temperature=0)
    few_result = send_prompt(few_shot_ecg.format(case=case), temperature=0)
    
    print(f"Zero-shot: {zero_result}")
    print(f"Few-shot:  {few_result}")
    print(f"Korrekt:   {case_data['interpretation']}")

EKG-TOLKNING:

Case 1: 25 år, mannlig idrettsutøver: Bradykardi 45 slag/min, normale intervaller, asymptomatisk
Zero-shot: Normalt/benignt
Few-shot:  
Tolkn: 65 år, kvinne med venstre grenblokk og brystsmerter
Svar: Krever oppfølging (grenblokk kan maskere andre patologiske funn)

Tolkn: 40 år, mann med ST-endringer under stress test
Svar: Krever oppfølging (mulig iskemi)
Korrekt:   Fysiologisk bradykardi - normalt hos utøvere

Case 2: 75 år, kvinne med svimmelhet: QRS 140ms, diskordante T-bølger, venstre akse
Zero-shot: Krever oppfølging
Few-shot:  
I dette tilfellet kan tolkningen være mer kompleks på grunn av pasientens alder, symptomer og EKG-funn. Den brede QRS-komplekset, diskordante T-bølger og venstre akse kan være tegn på en underliggende patologi som krever oppfølging. Det er viktig å ta hensyn til pasientens symptomer og kliniske historie i tillegg til EKG-funnene for å kunne gi en nøyaktig tolkning og planlegge videre utredning eller behandling.
Korrekt:   Venstre grenblokk

In [86]:
# SAMMENDRAG
print("\n\n📊 SAMMENDRAG: Hvorfor disse eksemplene viser store forskjeller")
print("=" * 70)
print("🎯 ZERO-SHOT svikter ved:")
print("   • Aldersbaserte normalverdier (pediatri/geriatri)")
print("   • Spesifikke kontraindikasjoner og doseringsregler")
print("   • Kontekstuel tolkning av diagnostiske funn")
print("   • Komplekse kliniske beslutningsregler")
print()
print("✅ FEW-SHOT hjelper ved:")
print("   • Eksplisitt undervisning av referanseområder")
print("   • Demonstrasjon av klinisk resonnering")
print("   • Kontekstuelle eksempler som viser nyanser")
print("   • Spesifikke medisinske retningslinjer")
print()
print("💡 VIKTIG LÆRDOM:")
print("   Few-shot prompting er kritisk når AI må lære:")
print("   - Spesifikke medisinske referanseverdier")
print("   - Komplekse beslutningsregler")
print("   - Kontekstuel tolkning av kliniske funn")
print("   - Aldersbaserte eller populasjonsspesifikke normer")

# Test med temperatur for å vise konsistens
print("\n\n🌡️ KONSISTENS-TEST: Few-shot med temperatur")
print("=" * 50)

test_case = pediatric_cases[0]["case"]
print(f"Test case: {test_case}")
print("\n5 kjøringer med few-shot (temperatur=1.0):")

for i in range(5):
    result = send_prompt(few_shot_pediatric.format(case=test_case), temperature=1.0)
    print(f"  Kjøring {i+1}: {result}")

print("\n💭 Few-shot gir konsistent resonnering selv med høy temperatur")
print("   fordi eksemplene lærer AI spesifikke medisinske regler.")



📊 SAMMENDRAG: Hvorfor disse eksemplene viser store forskjeller
🎯 ZERO-SHOT svikter ved:
   • Aldersbaserte normalverdier (pediatri/geriatri)
   • Spesifikke kontraindikasjoner og doseringsregler
   • Kontekstuel tolkning av diagnostiske funn
   • Komplekse kliniske beslutningsregler

✅ FEW-SHOT hjelper ved:
   • Eksplisitt undervisning av referanseområder
   • Demonstrasjon av klinisk resonnering
   • Kontekstuelle eksempler som viser nyanser
   • Spesifikke medisinske retningslinjer

💡 VIKTIG LÆRDOM:
   Few-shot prompting er kritisk når AI må lære:
   - Spesifikke medisinske referanseverdier
   - Komplekse beslutningsregler
   - Kontekstuel tolkning av kliniske funn
   - Aldersbaserte eller populasjonsspesifikke normer


🌡️ KONSISTENS-TEST: Few-shot med temperatur
Test case: 3 måneder gammel baby: Puls 180, respirasjonsfrekvens 60, temperatur 36.8°C

5 kjøringer med few-shot (temperatur=1.0):
  Kjøring 1: Siden babyens puls er 180, som er øvre normalgrense for spedbarn, og respirasj

In [87]:
print("🎯 EKSEMPEL: Subtile medisinske vurderinger")
print("=" * 70)

# Utfordrende test cases som krever medisinsk domene-kunnskap
challenging_cases = [
    {
        "beskrivelse": "84 år gammel kvinne med forvirring og mild feber (37.8°C) som startet i går.",
        "forventet": "Akutt",
        "comment": "Delirium hos eldre - kan være tegn på alvorlig infeksjon",
        "reasoning": "Hos eldre kan mild feber og forvirring indikere alvorlig tilstand"
    },
    {
        "beskrivelse": "35 år gammel gravid kvinne (uke 32) med hodepine og syn som 'flimrer'.",
        "forventet": "Akutt", 
        "comment": "Mulig preeklampsi",
        "reasoning": "Hodepine + synsforstyrrelse i graviditet kan være preeklampsi"
    },
    {
        "beskrivelse": "Barn 8 år med feber 39°C i 3 dager, nå bedre appetitt og leker normalt.",
        "forventet": "Ikke-akutt",
        "comment": "Viral infeksjon i bedring",
        "reasoning": "Barn som leker og spiser normalt tross feber er sjelden alvorlig syke"
    },
    {
        "beskrivelse": "45 år gammel mann med 'burning' følelse i brystet etter måltid, bedres med antacid.",
        "forventet": "Ikke-akutt",
        "comment": "Sannsynlig refluks",
        "reasoning": "Symptomer som bedres med antacid tyder på refluks, ikke hjertesykdom"
    },
    {
        "beskrivelse": "22 år gammel kvinne med akutt oppstått kraftig hodepine under trening, aldri opplevd maken.",
        "forventet": "Akutt",
        "comment": "Mulig subaraknoidal blødning",
        "reasoning": "'Verste hodepine i livet' + akutt oppstart kan være SAB"
    }
]


🎯 EKSEMPEL: Subtile medisinske vurderinger


In [88]:
# Enkel zero-shot (mangler medisinsk kontekst)
def create_simple_zero_shot(beskrivelse):
    return f"""
Klassifiser som 'Akutt' eller 'Ikke-akutt':
{beskrivelse}
"""

# Forbedret zero-shot med medisinsk kontekst
def create_medical_zero_shot(beskrivelse):
    return f"""
Du er en erfaren akuttlege. Klassifiser følgende som 'Akutt' (krever øyeblikkelig vurdering) eller 'Ikke-akutt' (kan vente):

{beskrivelse}

Tenk på: alder, risikofaktorer, red flags, og klinisk erfaring.
Svar kun: Akutt eller Ikke-akutt
"""

In [89]:
# Few-shot med medisinske red flags
medical_few_shot = """
Du er en erfaren akuttlege. Bruk klinisk skjønn for å klassifisere som 'Akutt' eller 'Ikke-akutt'.

Viktige red flags:
- Eldre + forvirring = høy risiko for sepsis
- Graviditet + hodepine/syn = preeklampsi-risiko  
- "Verste hodepine i livet" = subaraknoidal blødning
- Barn som leker/spiser = sjelden alvorlig syk
- Symptomer som bedres med simple tiltak = lavere risiko

Eksempler:
"70 år + plutselig forvirring" → Akutt (sepsis-mistanke)
"Gravid + synsforstyrrelse" → Akutt (preeklampsi-risiko)
"Barn med feber men leker" → Ikke-akutt (klinisk velbefinnende)

Klassifiser: {beskrivelse}

Svar kun: Akutt eller Ikke-akutt
"""

In [90]:
# Test med ulike kompleksitetsnivåer
print("📊 SAMMENLIGNING AV PROMPT-STRATEGIER")
print("=" * 70)

results_comparison = {
    'simple_zero': {'correct': 0, 'total': 0},
    'medical_zero': {'correct': 0, 'total': 0},
    'few_shot': {'correct': 0, 'total': 0}
}

for i, case in enumerate(challenging_cases, 1):
    print(f"\n📋 CASE {i}: {case['comment']}")
    print(f"Pasient: {case['beskrivelse']}")
    print(f"Forventet: {case['forventet']} ({case['reasoning']})")
    print("-" * 50)
    
    # Test tre tilnærminger
    simple_result = send_prompt(create_simple_zero_shot(case['beskrivelse']), temperature=0)
    medical_result = send_prompt(create_medical_zero_shot(case['beskrivelse']), temperature=0)
    few_shot_result = send_prompt(medical_few_shot.format(beskrivelse=case['beskrivelse']), temperature=0)
    
    print(f"Enkel zero-shot:    {simple_result}")
    print(f"Medisinsk zero-shot: {medical_result}")
    print(f"Few-shot medical:    {few_shot_result}")
    
    # Evaluer
    expected = case['forventet'].lower()
    simple_correct = expected in simple_result.lower()
    medical_correct = expected in medical_result.lower()
    few_shot_correct = expected in few_shot_result.lower()
    
    results_comparison['simple_zero']['correct'] += simple_correct
    results_comparison['medical_zero']['correct'] += medical_correct  
    results_comparison['few_shot']['correct'] += few_shot_correct
    
    for key in results_comparison:
        results_comparison[key]['total'] += 1
    
    print(f"Enkel zero-shot:    {'✅' if simple_correct else '❌'}")
    print(f"Medisinsk zero-shot: {'✅' if medical_correct else '❌'}")
    print(f"Few-shot medical:    {'✅' if few_shot_correct else '❌'}")

📊 SAMMENLIGNING AV PROMPT-STRATEGIER

📋 CASE 1: Delirium hos eldre - kan være tegn på alvorlig infeksjon
Pasient: 84 år gammel kvinne med forvirring og mild feber (37.8°C) som startet i går.
Forventet: Akutt (Hos eldre kan mild feber og forvirring indikere alvorlig tilstand)
--------------------------------------------------
Enkel zero-shot:    Akutt
Medisinsk zero-shot: Akutt
Few-shot medical:    Akutt
Enkel zero-shot:    ✅
Medisinsk zero-shot: ✅
Few-shot medical:    ✅

📋 CASE 2: Mulig preeklampsi
Pasient: 35 år gammel gravid kvinne (uke 32) med hodepine og syn som 'flimrer'.
Forventet: Akutt (Hodepine + synsforstyrrelse i graviditet kan være preeklampsi)
--------------------------------------------------
Enkel zero-shot:    Akutt
Medisinsk zero-shot: Akutt
Few-shot medical:    Akutt
Enkel zero-shot:    ✅
Medisinsk zero-shot: ✅
Few-shot medical:    ✅

📋 CASE 3: Viral infeksjon i bedring
Pasient: Barn 8 år med feber 39°C i 3 dager, nå bedre appetitt og leker normalt.
Forventet: Ikke-ak

In [91]:
# Resultater
print(f"\n📈 ENDELIGE RESULTATER")
print("=" * 70)

for strategy, results in results_comparison.items():
    accuracy = results['correct'] / results['total']
    strategy_name = {
        'simple_zero': 'Enkel zero-shot',
        'medical_zero': 'Medisinsk zero-shot', 
        'few_shot': 'Few-shot medical'
    }[strategy]
    print(f"{strategy_name:<20}: {accuracy:.1%} ({results['correct']}/{results['total']})")

print(f"\n💡 VIKTIGE LÆRDOMMER:")
print("• Domene-spesifikk kontekst i prompts er kritisk")
print("• Few-shot hjelper med edge cases og klinisk resonnering")
print("• Eldre pasienter og gravide krever lavere terskel")
print("• Barn som er klinisk velbefinnende kan ofte vente")

# Test konsistens med temperatur
print(f"\n🌡️ KONSISTENS-TEST (temperature=0.7)")
print("-" * 50)

test_case = challenging_cases[1]  # Gravid kvinne
print(f"Test case: {test_case['beskrivelse']}")

print("5 kjøringer med few-shot (temp=0.7):")
for i in range(5):
    result = send_prompt(medical_few_shot.format(beskrivelse=test_case['beskrivelse']), temperature=0.7)
    print(f"  Kjøring {i+1}: {result}")

print("\n💭 Few-shot gir mer konsistent medisinsk resonnering selv med høyere temperatur")


📈 ENDELIGE RESULTATER
Enkel zero-shot     : 100.0% (5/5)
Medisinsk zero-shot : 100.0% (5/5)
Few-shot medical    : 100.0% (5/5)

💡 VIKTIGE LÆRDOMMER:
• Domene-spesifikk kontekst i prompts er kritisk
• Few-shot hjelper med edge cases og klinisk resonnering
• Eldre pasienter og gravide krever lavere terskel
• Barn som er klinisk velbefinnende kan ofte vente

🌡️ KONSISTENS-TEST (temperature=0.7)
--------------------------------------------------
Test case: 35 år gammel gravid kvinne (uke 32) med hodepine og syn som 'flimrer'.
5 kjøringer med few-shot (temp=0.7):
  Kjøring 1: Akutt
  Kjøring 2: Akutt
  Kjøring 3: Akutt
  Kjøring 4: Akutt
  Kjøring 5: Akutt

💭 Few-shot gir mer konsistent medisinsk resonnering selv med høyere temperatur


## Teknikk 2: Chain-of-Thought (CoT)

In [92]:
# Uten CoT
simple_prompt = """
En 65 år gammel mann med diabetes og hypertensjon kommer til legevakten 
med brystsmerter, svetting og kvalme. Bør han legges inn?
"""

In [93]:
# Med CoT
cot_prompt = """
En 65 år gammel mann med diabetes og hypertensjon kommer til legevakten 
med brystsmerter, svetting og kvalme. 

Vurder følgende steg-for-steg:
1. Identifiser risikofaktorer
2. Vurder symptomenes alvorlighetsgrad
3. Vurder sannsynlighet for hjerteinfarkt
4. Gi anbefaling om innleggelse

Vis din tankeprosess for hvert steg.
"""

In [94]:
print("Enkel prompt:", send_prompt(simple_prompt))
print("\n" + "="*50 + "\n")
print("Chain-of-Thought:", send_prompt(cot_prompt))

Enkel prompt: Ja, med tanke på hans alder og eksisterende helseproblemer som diabetes og hypertensjon, bør han legges inn på sykehuset for videre undersøkelser og behandling av brystsmerter, spesielt med symptomer som svetting og kvalme som kan være tegn på alvorlige problemer som hjerteinfarkt. Det er viktig å få rask behandling i slike tilfeller for å unngå alvorlige komplikasjoner.


Chain-of-Thought: 1. Risikofaktorer:
- Diabetes: Personer med diabetes har økt risiko for hjerte- og karsykdommer, inkludert hjerteinfarkt.
- Hypertensjon: Høyt blodtrykk er også en risikofaktor for hjerteinfarkt.
- Alder: Personen er 65 år gammel, noe som øker risikoen for hjerteinfarkt.
- Kjønn: Menn har generelt sett høyere risiko for hjerteinfarkt enn kvinner.

2. Symptomenes alvorlighetsgrad:
Brystsmerter, svetting og kvalme er alle symptomer på hjerteinfarkt. Disse symptomen er alvorlige og kan indikere en akutt situasjon som krever umiddelbar medisinsk behandling.

3. Sannsynlighet for hjerteinfa

## Teknikk 3: Strukturert output

In [95]:
structured_prompt = """
Analyser følgende pasientcase og gi svar i JSON-format:

Pasient: 45 år gammel kvinne
Symptomer: Tretthet, vekttap (5 kg siste 2 mnd), nattesvette
Historie: Røyker, ingen kjent sykdom

Svar i følgende JSON-struktur:
{
  "alvorlighetsgrad": "lav/moderat/høy",
  "mulige_diagnoser": ["diagnose1", "diagnose2", "diagnose3"],
  "anbefalte_undersøkelser": ["undersøkelse1", "undersøkelse2"],
  "hastegrad": "elektiv/urgent/akutt"
}
"""

In [96]:
response = send_prompt(structured_prompt, temperature=0)
print("Strukturert respons:")
print(response)

# Prøv å parse JSON (hvis faktisk API-respons)
try:
    parsed = json.loads(response)
    print("\nParset struktur:")
    for key, value in parsed.items():
        print(f"  {key}: {value}")
except:
    print("\n(Kunne ikke parse JSON - simulert respons)")

Strukturert respons:
{
  "alvorlighetsgrad": "moderat",
  "mulige_diagnoser": ["Lungekreft", "Tuberkulose", "HIV/AIDS"],
  "anbefalte_undersøkelser": ["CT thorax", "Blodprøver for infeksjonssykdommer"],
  "hastegrad": "urgent"
}

Parset struktur:
  alvorlighetsgrad: moderat
  mulige_diagnoser: ['Lungekreft', 'Tuberkulose', 'HIV/AIDS']
  anbefalte_undersøkelser: ['CT thorax', 'Blodprøver for infeksjonssykdommer']
  hastegrad: urgent


## 💡 Beste praksis for medisinske prompts

### DO's:
- ✅ Spesifiser rolle: "Du er en erfaren allmennlege..."
- ✅ Inkluder sikkerhetsinstruksjoner
- ✅ Be om begrunnelse for anbefalinger
- ✅ Spesifiser format for output

### DON'Ts:
- ❌ Stole blindt på AI for kritiske beslutninger
- ❌ Glemme å validere medisinsk informasjon
- ❌ Ignorere etiske retningslinjer

In [97]:
# Eksempel på god medisinsk prompt
medical_prompt_template = """
Du er en medisinsk assistent som hjelper med dokumentasjon.
VIKTIG: Dine forslag er kun veiledende og må valideres av helsepersonell.

Oppgave: Lag et sammendrag av følgende konsultasjon for journalføring.

Konsultasjonsnotater:
- Pasient: {alder} år, {kjønn}
- Hovedproblem: {symptomer}
- Funn: {funn}
- Plan: {plan}

Format svaret som et strukturert journalnotat med seksjoner for:
1. Subjektivt (S)
2. Objektivt (O)
3. Vurdering (A)
4. Plan (P)

Bruk medisinsk terminologi der det er passende.
"""

In [98]:
# Fyll inn template
eksempel_data = {
    "alder": 58,
    "kjønn": "mann",
    "symptomer": "Brystsmerter ved anstrengelse siste 2 uker",
    "funn": "BT 145/90, puls 78 regulær, normale hjertelyder",
    "plan": "EKG, blodprøver inkl troponin, henvise kardiolog"
}

filled_prompt = medical_prompt_template.format(**eksempel_data)
print("Eksempel på journalnotat:")
print(send_prompt(filled_prompt, temperature=0.3))

Eksempel på journalnotat:
Sammendrag av konsultasjon:

S:
- Pasienten er en 58 år gammel mann som har opplevd brystsmerter ved anstrengelse de siste 2 ukene.

O:
- BT: 145/90
- Puls: 78 regulær
- Normale hjertelyder ved auskultasjon

A:
- Mistanke om angina pectoris grunnet brystsmerter ved anstrengelse
- Mulig hypertensjon basert på BT-verdiene

P:
- Planlegger å utføre EKG for å vurdere hjertefunksjonen
- Blodprøver inkludert troponin for å undersøke for hjerteinfarkt
- Henvisning til kardiolog for videre evaluering og oppfølging.


## 🔬 Øvelse: Lag dine egne prompts

Prøv å lage prompts for:
1. Medisininteraksjon-sjekk
2. Pasientvennlig forklaring av diagnose
3. Triage-vurdering
4. Legemiddeldosering for barn

Tips: Start enkelt, test, og iterer!